In [ ]:
# I used ChatGTP 3.5 and 4 for assistance

In [1]:
!pip install git+https://github.com/tensorflow/privacy.git@master

  Cloning https://github.com/tensorflow/privacy.git (to revision master) to /tmp/pip-req-build-j1mgi4_z
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/privacy.git /tmp/pip-req-build-j1mgi4_z
  Resolved https://github.com/tensorflow/privacy.git to commit f27d79d505e3734ec38df688666d48f025132b92
  Preparing metadata (setup.py) ... done


In [2]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [3]:
!pip install tensorflow-privacy==0.8.12 --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-privacy
    Found existing installation: tensorflow_privacy 0.9.0
    Uninstalling tensorflow_privacy-0.9.0:
      Successfully uninstalled tensorflow_privacy-0.9.0


In [4]:
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

In [5]:
from tensorflow.keras import layers

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Install DVC and Google Drive support
!pip install dvc[gdrive]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.5/458.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8

In [9]:
# Load and preprocess data_v2.csv

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%cd /content/drive/MyDrive/ML_OPS/
!dvc pull

/content/drive/MyDrive/ML_OPS
Fetching
!
  0% |          |0/? [00:00<?,    ?files/s]
 50% 1/2 [00:01<00:01,  1.96s/files{'info': ''}]
100% 2/2 [00:02<00:00,  1.01files/s{'info': ''}]
Fetching
Building workspace index          |3.00 [00:01, 1.53entry/s]
Comparing indexes          |4.00 [00:00, 1.49kentry/s]
Applying changes          |0.00 [00:00,     ?file/s]
Everything is up to date.


In [18]:
df = pd.read_csv("data_v2.csv")
df.shape

(30029, 14)

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [42]:
# Feature engineering
df["total_lift"] = df["deadlift"] + df["candj"] + df["snatch"] + df["backsq"]
X = df.drop(columns=["total_lift"]).select_dtypes(include="number").dropna()
y = df["total_lift"].loc[X.index]

# Train-test split + scaling
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
# Train Non-DP Model
# Non-DP model with NO L2
model_nodp = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

model_nodp.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('model_nodp.h5', monitor='val_loss', save_best_only=True)

history_nodp = model_nodp.fit(X_train_scaled, y_train, batch_size=64, epochs=30, verbose=0)

y_pred_nodp = model_nodp.predict(X_test_scaled).flatten()

188/188 [==============================] - 0s 2ms/step


In [52]:
# Train DP Model

from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

dp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

dp_optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=1.0,
    noise_multiplier=5.0,
    num_microbatches=64,
    learning_rate=0.15
)

dp_model.compile(
    optimizer=dp_optimizer,
    loss=tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE),
    metrics=['mse']
)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train))
train_dataset = train_dataset.shuffle(1024).batch(64, drop_remainder=True)

history_dp = dp_model.fit(train_dataset, epochs=30, verbose=0)

y_pred_dp = dp_model.predict(X_test_scaled).flatten()

188/188 [==============================] - 0s 1ms/step


In [53]:
# Evaluate Both Models
def evaluate(y_true, y_pred, X_test):
    mse = mean_squared_error(y_true, y_pred)
    mae = np.mean(np.abs(y_pred - y_true))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    adj_r2 = 1 - (1-r2)*(len(y_true)-1)/(len(y_true)-X_test.shape[1]-1)
    mbd = np.mean(y_pred - y_true)

    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"MAPE: {mape}")
    print(f"R2: {r2}")
    print(f"Adjusted R2: {adj_r2}")
    print(f"MBD: {mbd}")
    return mae, mse, r2

print("\n📊 Non-DP Model:")
evaluate(y_test, y_pred_nodp, X_test_scaled)

print("\n🔐 DP Model:")
evaluate(y_test, y_pred_dp, X_test_scaled)


📊 Non-DP Model:
MAE: 61.7313074892535
MSE: 3979.1989000424483
MAPE: 0.06579326288142374
R2: 0.9481935606083278
Adjusted R2: 0.9481330996087043
MBD: -61.60608693801519

🔐 DP Model:
MAE: 98.88870163548204
MSE: 11690.89842298688
MAPE: 0.12337290792854369
R2: 0.8477925241238369
Adjusted R2: 0.8476148895237814
MBD: -90.73006456739061


(98.88870163548204, 11690.89842298688, 0.8477925241238369)

In [54]:
# Compute DP (ε, δ)
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy_lib

epsilon, _ = compute_dp_sgd_privacy_lib.compute_dp_sgd_privacy(
    n=X_train_scaled.shape[0],
    batch_size=batch_size,
    noise_multiplier=1.1,
    epochs=30,
    delta=1e-5
)

print(f"\n🔐 DP-SGD Privacy Guarantee: (ε, δ) = ({epsilon}, 1e-5)")


🔐 DP-SGD Privacy Guarantee: (ε, δ) = (1.3774644037710095, 1e-5)


**Conclusion:**

As expected, the **DP model performs worse** than the Non-DP model:

* 📉 **Higher MAE & MSE**: Indicates more prediction error due to DP noise.
* 📉 **Lower R² (0.85 vs 0.95)**: DP model explains less variance.
* 🔐 This confirms that **differential privacy degrades model performance**, which is the tradeoff for stronger privacy.
